#### Challenge 49: CBC-MAC Message Forgery

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import math
import base64
import cryptopals as cp

from decimal import *

import pdb

<div class="alert alert-block alert-info">   

Let's talk about CBC-MAC. 
    
CBC-MAC is like this: 
    
1. Take the plaintext `P`.
2. Encrypt `P` under CBC with key `K`, yielding ciphertext `C`.
3. Chuck all of `C` but the last block `C[n]`.
4. `C[n]` is the MAC.
    
</div>

In [2]:
# ---------UNKNOWN PARAMETERS ------------
K_Shared = random.Random.get_random_bytes(32)
# ---------END UNKNOWN PARAMETERS---------

IV = b'\x00' * 16
test_msg = b'Some Test Message That\'s Longer than one block'

In [3]:
def CBC_MAC(p, K, IV):
    
    a_out = cp.AESEncrypt(p, K, 'CBC', IV)
    return(a_out[-16:])

<div class="alert alert-block alert-info">   

Suppose there's an online banking application, and it carries out user requests by talking to an API server over the network. Each request looks like this: 

`message || IV || MAC`
    
The message looks like this: 
    
`from=#{from_id}&to=#{to_id}&amount=#{amount}`
    
Now, write an API server and a web frontend for it. (NOTE: No need to get ambitious and write actual servers and web apps. Totally fine to go lo-fi on this one.) The client and server should share a secret key `K` to sign and verify messages. 
    
The API server should accept messages, verify signatures, and carry out each transaction if the MAC is valid. It's also publicly exposed - the attacker can submit messages freely assuming he can forge the right MAC. 
    
The web client should allow the attacker to generate valid messages for accounts he controls. (Feel free to sanitize params if you're feeling anal-retentive.) Assume the attacker is in a position to capture and inspect messages from the client to the API server. 
    
</div>

The way I interpret this challenge, based on this first stage of the attack, only the first block of the message is being manipulated. Therefore, to get it to work you have to make some big assumptions:

1.  The to_id will have to appear in the first block (this means the account #'s have to be short enough for that to happen), and 
2.  We have intercepted a previous valid transaction from the target account owner to someone else in the amount we want to steal.  


In [4]:
class account:
    
    def __init__(self, starting_balance):
            
        #self.id = id
        self.balance = starting_balance
        self.key = random.Random.get_random_bytes(32)
        
    def __str__(self):
        
        display_str = ''
        display_str += f'Account Key:  {self.key}\n'
        display_str += f'Account Balance: ${self.balance:,.2f}\n'
        return (display_str)
        
class bank_app:

    next_id = 1
    
    def __init__(self):
        
        # Create some test accounts
        self.accounts = {}
        
        self.accounts[self.next_id] = account(5000000)
        self.next_id += 1
        self.accounts[self.next_id] = account(300000)
        self.next_id += 1
        self.accounts[self.next_id] = account(70230.24)
        self.next_id += 1
    
    def __str__(self):
        
        display_str = ''
        for account in self.accounts:
            display_str += 'Account ID:  ' + str(account) + '\n'
            display_str += str(self.accounts[account])
            display_str += '\n'
            
        return(display_str)
    
    def open_new_account(self, deposit_amount):
        
        id = self.next_id
        self.accounts[id] = account(deposit_amount)
        self.next_id += 1
        
        return(id)
    
    def process_msg(self, msg):
        
        if len(msg) < 32:
            raise(ValueError('Invalid message)'))
        
        IV, MAC = msg[-32:-16], msg[-16:]
        transaction = cp.strip_PKCS7_pad(msg[:-32]).split(b'&')
        
        if transaction[0][:5] == b'from=' and \
           transaction[1][:3] == b'to=' and \
           transaction[2][:7] == b'amount=':
            
            start = transaction[0].find(b'=') + 1
            from_id = int(transaction[0][start:])            
            key = self.accounts[from_id].key        
            myMAC = CBC_MAC(msg[:-32], key, IV)       
            
        else:
        
            raise(Exception('Invalid transaction format'))
                    
        if (MAC == myMAC):            
            
            start = transaction[1].find(b'=') + 1
            to_id = int(transaction[1][start:])            

            #print(transaction[2])
            start = transaction[2].find(b'=') + 1
            amount = float(transaction[2][start:])     

        
        if self.accounts[from_id].balance >= amount:
            self.accounts[from_id].balance -= amount
            self.accounts[to_id].balance += amount
            return(True)
        else:
            return(False)    

In [5]:
def create_signed_transaction(from_id, to_id, amount):
    
    # To simplify things, just assuming the authentic user has access to their account transaction key.
    id_key = my_bank.accounts[from_id].key
    msg = cp.PKCS7_pad((f'from={from_id}&to={to_id}&amount={amount}').encode())
    IV = b'\x00'*16
    MAC = CBC_MAC(msg, id_key, IV)
    
    return(msg + IV + MAC)
    

In [6]:
my_bank = bank_app()

In [7]:
print('*****************************')
print('Balances before transfer')
print('*****************************\n')
print(my_bank)

print('Transfering $1000000.00 from Acct 10001 to Acct 10002')
t = create_signed_transaction(1, 2, 1000000)
print()
print('*****************************')
print('Balances after transfer')
print('*****************************\n')
my_bank.process_msg(t)
print(my_bank)

*****************************
Balances before transfer
*****************************

Account ID:  1
Account Key:  b"\x8dp\xa3\xbaT\x8e:\xf3e\x92\xd7\xb54\xf2\xe8\xb1\xea$\xedE\xc7D\x1bm\xa2\xfa'\xc9\x07d14"
Account Balance: $5,000,000.00

Account ID:  2
Account Key:  b'N\x98\xc3P\xdd\x8f\x13\xbc\r#4\x1aa\x15\xab\xbc#\xb6\xac)\xe1\xf549L-\x89\xcd\x8e\xc4W\xf5'
Account Balance: $300,000.00

Account ID:  3
Account Key:  b'\xde\x86\xe0\x03\xb1&`s\xdclg\xfc~\x1e\xc0\x11\xe6\x9d!\x16Rp\xa5!\x0f\xea\xb6\x83#\x18\xe9\xaa'
Account Balance: $70,230.24


Transfering $1000000.00 from Acct 10001 to Acct 10002

*****************************
Balances after transfer
*****************************

Account ID:  1
Account Key:  b"\x8dp\xa3\xbaT\x8e:\xf3e\x92\xd7\xb54\xf2\xe8\xb1\xea$\xedE\xc7D\x1bm\xa2\xfa'\xc9\x07d14"
Account Balance: $4,000,000.00

Account ID:  2
Account Key:  b'N\x98\xc3P\xdd\x8f\x13\xbc\r#4\x1aa\x15\xab\xbc#\xb6\xac)\xe1\xf549L-\x89\xcd\x8e\xc4W\xf5'
Account Balance: $1,300,000.00



<div class="alert alert-block alert-info">   

One thing we haven't discussed is the `IV`. Assume the client generates a per-message IV and sends it along with the MAC. That's how CBC works, right? 
Wrong. 
    
For messages signed under CBC-MAC, an attacker-controlled `IV` is a liability. Why? Because it yields full control over the first block of the message. 
Use this fact to generate a message transferring 1M spacebucks from a target victim's account into your account. 
    
I'll wait. Just let me know when you're done. 
... waiting 
... waiting 
... waiting 
All done? Great - I knew you could do it! 

</div>

In [8]:
# Open a new $0 balance account that I control and have the key for. 
my_id = my_bank.open_new_account(0)
my_key = my_bank.accounts[my_id].key

In [9]:
intercepted_msg = t
malicious_message = f'from=1&to={my_id}&amou'.encode()
intercepted_IV = intercepted_msg[-32:-16]
forged_IV = cp.bitwise_xor(cp.bitwise_xor(intercepted_msg[0:16], intercepted_IV), malicious_message)
my_evil_transaction = malicious_message + intercepted_msg[16:-32] + \
                      forged_IV + intercepted_msg[-16:]   
print(f'Balance before attack: ${my_bank.accounts[my_id].balance:,.2f}')
my_bank.process_msg(my_evil_transaction)
print(f'Balance after attack: ${my_bank.accounts[my_id].balance:,.2f}')

Balance before attack: $0.00
Balance after attack: $1,000,000.00


<div class="alert alert-block alert-info">   

Now let's tune up that protocol a little bit. 
    
As we now know, you're supposed to use a fixed `IV` with CBC-MAC, so let's do that. We'll set ours at `0` for simplicity. This means the `IV` comes out of the protocol: 
    
`message || MAC`
    
Pretty simple, but we'll also adjust the message. For the purposes of efficiency, the bank wants to be able to process multiple transactions in a single request. So the message now looks like this: 
    
`from=#{from_id}&tx_list=#{transactions}`
    
With the transaction list formatted like: 
    
`to:amount(;to:amount)*`

</div>

In [15]:
class new_bank(bank_app):
    
    def process_msg(self, msg):
                
        if len(msg) < 32:
            raise(ValueError('Invalid message)'))
        
        IV = b'\x00' * 16
        MAC = msg[-16:]
        data = cp.strip_PKCS7_pad(msg[:-16])
        transaction = data.split(b'&')
        
        if transaction[0][:5] == b'from=' and \
           transaction[1][:8] == b'tx_list=':
            
            start = transaction[0].find(b'=') + 1
            from_id = int(transaction[0][start:])            
            key = self.accounts[from_id].key       
            #pdb.set_trace()
            myMAC = CBC_MAC(msg[:-16], key, IV)       
            
        else:
        
            raise(Exception('Invalid transaction format'))
                    
        if (MAC != myMAC):            
            return(False)
        
        start = transaction[1].find(b'=') + 1
        transaction_list = transaction[1][start:].split(b';')           

        for transfer in transaction_list:
            to_id, amount = transfer.split(b':')
            to_id = int(to_id)
            amount = float(amount)
            if self.accounts[from_id].balance >= amount:
                self.accounts[from_id].balance -= amount
                self.accounts[to_id].balance += amount
            else:
                return(False)    
        
        return(True)
def new_create_signed_transaction(from_id, transfer_list):
    
    # To simplify things, just assuming the authentic user has access to their account transaction key.
    id_key = my_new_bank.accounts[from_id].key
    msg = f'from={from_id}&tx_list='
    for transfer in transfer_list:
        msg += f'{transfer[0]}:{transfer[1]};'
    msg = cp.PKCS7_pad(msg[:-1].encode())
    IV = b'\x00'*16
    MAC = CBC_MAC(msg, id_key, IV)
    
    return(msg + MAC)

In [16]:
my_new_bank = new_bank()
print(my_new_bank)

from_id = 1
transfer_list = [[2, 100],[3, 1001.99]]
transaction = new_create_signed_transaction(from_id, transfer_list)
my_new_bank.process_msg(transaction)

print(my_new_bank)

Account ID:  1
Account Key:  b'\x0f\xa5\xdfm1\x11\xea\xab\x97=\xc4\x02d\x0b\xe4\xdb\xeb\x0b\xaad\xe2\xd6\xaf\xf6\xcb\x1c\xaf\x80>.W\x83'
Account Balance: $5,000,000.00

Account ID:  2
Account Key:  b'\x0e5\x11\xfd\x85oHyP\x0fR}\xd7\xb6\xfe\xb7f\n\x03\x81n\xfbr\xac\xeb\x0cTy\x1d\xd5\xacy'
Account Balance: $300,000.00

Account ID:  3
Account Key:  b'1k\xde\x88z\x80L\xde\xae\xf4\x15\xf2\xda\xd3\xb6\x10\xe1LN\xc9\xab\x13\xcd\xdf0V\x0e\x80\xac\xadV\xa7'
Account Balance: $70,230.24


Account ID:  1
Account Key:  b'\x0f\xa5\xdfm1\x11\xea\xab\x97=\xc4\x02d\x0b\xe4\xdb\xeb\x0b\xaad\xe2\xd6\xaf\xf6\xcb\x1c\xaf\x80>.W\x83'
Account Balance: $4,998,898.01

Account ID:  2
Account Key:  b'\x0e5\x11\xfd\x85oHyP\x0fR}\xd7\xb6\xfe\xb7f\n\x03\x81n\xfbr\xac\xeb\x0cTy\x1d\xd5\xacy'
Account Balance: $300,100.00

Account ID:  3
Account Key:  b'1k\xde\x88z\x80L\xde\xae\xf4\x15\xf2\xda\xd3\xb6\x10\xe1LN\xc9\xab\x13\xcd\xdf0V\x0e\x80\xac\xadV\xa7'
Account Balance: $71,232.23




<div class="alert alert-block alert-info">   

There's still a weakness here: the MAC is vulnerable to length extension attacks. How? 
    
Well, the output of CBC-MAC is a valid `IV` for a new message. 
    
_"But we don't control the `IV` anymore!"_
    
With sufficient mastery of CBC, we can fake it. 
    
Your mission: capture a valid message from your target user. Use length extension to add a transaction paying the attacker's account 1M spacebucks. 

<div class="alert alert-block alert-warning">    

#### **Hint!**
    
This would be a lot easier if you had full control over the first block of your message, huh? Maybe you can simulate that. 
    
</div>
    
_Food for thought: How would you modify the protocol to prevent this?_
    
</div>    

---

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)